# Bot Skeleton

The next cell will authenticate your bot based on your keys and tokens. See the [Tweepy tutorial](https://github.com/comp-journalism/UMD-J479V-J779V-Spring2016/blob/master/Weekly/Week_5/Tweepy-skeleton.ipynb) for more details. 

In [38]:
import tweepy

#Setup and authenticate Tweepy
CONSUMER_KEY = "EoCgZlaz4kbuX6qaWfPhkWQJs"
CONSUMER_SECRET = "aYZvsKONPT7LwS8iQgBbowKNIlnIbQtwXQPyqtxcYqCgZ0rwDY"
ACCESS_TOKEN = "758422215733944320-rlMTPLdz6em9AegePS6jV7145YC7zny"
ACCESS_TOKEN_SECRET = "RN6x2AMQhjYTKURhZ3VDGxvC5Hr2y8oonzi4kkPCozE2d"

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)



taysvea_tweets = api.user_timeline('taysvea')
for tweet in taysvea_tweets:
    print( tweet.created_at, tweet.text)
    
api.update_status(status = "Tweet Test")


2016-07-29 17:39:29 taylorgandolfitest
2016-07-27 12:39:34 That Time 2 Bots Were Talking, and Bank of America Butted In https://t.co/79WcjCsHXH
2016-07-22 21:00:24 That crazy moment when I was in Munich shopping center 4 days ago...
2016-07-12 22:11:13 Magic Leap wants the entire world to be your computer display via @FortuneMagazine https://t.co/n7brqBSwXi
2016-07-12 21:42:44 Magic Leap wants the entire world to be your computer display via @FortuneMagazine https://t.co/n7brqCa8lS
2016-07-07 19:26:21 RT @uxpin: “A user interface is like a joke. If you have to explain it, it’s not that good” –Martin LeBlanc, Iconfinder #design https://t.c…
2016-07-07 03:58:00 Welcome to the Bulls Wade 😜🏀
2016-06-16 17:19:27 Russian Robot Escapes Its Lab, Hijinks Ensue - ​A robot escaped from a testing facility in Perm and https://t.co/NOaJBGJuRj via @PopMech
2016-06-14 15:08:41 Request for @Disneyland for an attraction with 101 Dalmatians where you sit in room with 101 Dalmatian puppies and play #milli

TweepError: [{'message': 'Status is a duplicate.', 'code': 187}]

The next cell has a number of essential functions that allow your bot to listen to the twitter stream and then respond. The assignment involves fleshing out the editorial logic of the bot and writing code in the sections marked "TODO".

In [39]:
import random
import json
import time
import pprint


# If debug_mode is True then the bot won't actually tweet. Set debug_mode = False for it to tweet. 
debug_mode = False


user = api.get_user(screen_name="taysvea")
pprint.pprint(user._json)
print ("Followers:",user.followers_count)
print ("Friends:",user.friends_count)


# Functions that allow the bot to tweet or reply to tweets
def tweet(status):
    print ("JUST TWEETED: "), status
    # Only *actually* send the tweet on twitter if we're not in debug mode
    if debug_mode == False:
        api.update_status(status)
        
def tweet_with_probability(status, probability):
    # Change the probability of tweeting variable to affect how often the bot tweets
    rand = random.random()
    if rand <= probability:
        print ("JUST TWEETED: "), status
        # Only *actually* send the tweet on twitter if we're not in debug mode
        if debug_mode == False:
            api.update_status(status)
        
def tweet_reply(status, tweet_to_reply_to):
    screen_name = tweet_to_reply_to["user"]["screen_name"]
    at = ".@"+screen_name
    reply_to_id = tweet_to_reply_to["id"]
    status = at + " " + status
    print ("JUST TWEETED: Hello World"), status
    # To actually have the bot tweet a status message uncomment the next line, it will send the tweet in response to the appropriate tweet. 
    # Only *actually* send the tweet on twitter if we're not in debug mode
    if debug_mode == False:
        api.update_status(status=status, in_reply_to_status_id=reply_to_id)
    
# We derive a class that is used to listen to the twitter stream
class TwitterStreamListener(tweepy.StreamListener):
    def __init__(self):
        super(TwitterStreamListener, self).__init__()
        self.num_tweets = 0 
        self.api_count = 0
        self.old_time = time.time()
    
    # Everytime the listener encounters a tweet that matches its filters it will trigger the on_data function
    def on_data(self, data):
        try:
            print ("a tweet found")
            # Increment out counter of tweets
            self.num_tweets = self.num_tweets + 1
               
            
            # The data variable represents the tweet that was detected by the stream listener
            # Here we just parse it as JSON and put it in another variable. 
            tweet_data = json.loads(data)
            
            print(tweet_data)
            # Make sure the bot never interacts with itself
            user = tweet_data["user"]["screen_name"]
            
            user_mentions = tweet_data['entities']['user_mentions']
            print("got here")
            user_mentions_list = []
            for u in user_mentions:
                print("iterating user mentions")
                print(u)
                user_mentions_list.append(u["screen_name"])
            print("got past loop")
            if user == "syrianrefugeeb1" or "syrianrefugeeb1" in user_mentions_list:
                print ("dont talk to me")
                # return without doing anything
                return
      
    

            
            # TODO: Add your editorial logic here, some example filters are below
            
            # Possible filters
            # Maybe you don't want to respond to someone's RT
            #if tweet_data["retweeted"] == True
            #    return
            
            # Maybe you don't want to respond to a tweet without a URL
            #if len(tweet_data["entities"]["urls"]) == 0:
            #    return
            
            # Maybe you only want to do something with every Nth tweet since otherwise the bot is tweeting too frequently
            # e.g. Don't do anything unless this is the 5th tweet you've encountered
            if self.num_tweets == 5:
                print ("5")
                self.bot_action(tweet_data)
            
        except Exception as e:
           
            print ("test exception: ") + e
            
            pass
        
        return True
    
    def api_limit_checker(self):
        # API limits are reset every 15 minutes (900 seconds). This checks if we are outside a 15 min window 
        if time.time() - self.old_time > 900: 
            # Reset the timer of window since new window started & Twitter limits refreshed
            self.old_time = time.time()
            #Reset the api counter to 0 since new window started & Twitter limits refreshed
            self.api_count = 0
        else:
            # The bot cannot tweet more than 140 times every 15 minutes (and you should REALLY consider not ever getting close to that as you may trigger Twitters spam detectors)
            # The threshold below is set much more conservatively. It will only let you tweet 5 times in a 15 minute window
            if self.api_count > 5:
                print ("Taking a break")
                # We put the code to sleep for 15-x minutes where x is the current time - old time (when the window started)... So if within 6 minutes we tweet more than 140 times, then the script will halt for the next 15-6=9 minutes until a new window starts
                time.sleep(900 - (time.time() - self.old_time))
                # Since we have slept through the remainder, a new window has started and we reset the API Count as well
                self.api_count = 0
                #Similarly as above, we reset the timer as well since new window has started 
                self.old_time = time.time()
                
    def bot_action(self, tweet_to_reply_to):
        print ("bot action")
        # NOTE: if you'd like your bot to reply, user the tweet_data variable above as the tweet_to_reply_to parameter for this function
        # invoke the API limit checker code, whcih will put bot to sleep if it's too active
        self.api_limit_checker()
        
        # reset the num_tweets counter
        self.num_tweets = 20
        
        print("got to here in bot action")
        # Actually tweet something, just a dummy output right now
        # TODO: Write code that constructs a text to tweet
        try:
            print(tweet_to_reply_to)
            api.update_status('Hello World',tweet_to_reply_to["id"]) 
        except Exception as e:
            print("Exception in bot action")
            print(e)
       
        print("we think we did something")
        tweet("Hello world")
        
        # Increment API count
        self.api_count = self.api_count + 1
        
        return 
    
    def on_error(self, status):
        print (status)


def listen():
    print ("listening")
    listener = TwitterStreamListener()
    stream = tweepy.Stream(auth, listener)
    # track_object filters the twitter stream to capture data, can provide a hashtag, word, or screenname amongths other things. See documentation at: https://dev.twitter.com/streaming/overview/request-parameters
    # TODO: update track_object based on what you want your bot to listen for
    track_object = ["kim kardashian"]
    
    try:
        # Start filtering the twitter stream
        stream.filter(track=track_object)
        print ("tried")
    except:
        # do nothing in the event of an error
        pass
    return

{'contributors_enabled': False,
 'created_at': 'Thu Jul 01 17:35:59 +0000 2010',
 'default_profile': False,
 'default_profile_image': False,
 'description': 'prototyper, dog lover, scuba diver :)',
 'entities': {'description': {'urls': []},
              'url': {'urls': [{'display_url': 'taylorgandolfi.com',
                                'expanded_url': 'http://taylorgandolfi.com',
                                'indices': [0, 23],
                                'url': 'https://t.co/WEwcwfqTHv'}]}},
 'favourites_count': 903,
 'follow_request_sent': False,
 'followers_count': 630,
 'following': False,
 'friends_count': 1238,
 'geo_enabled': True,
 'has_extended_profile': False,
 'id': 161750294,
 'id_str': '161750294',
 'is_translation_enabled': False,
 'is_translator': False,
 'lang': 'en',
 'listed_count': 13,
 'location': 'Miami, FL',
 'name': 'Taylor Gandolfi',
 'notifications': False,
 'profile_background_color': '000000',
 'profile_background_image_url': 'http://pbs.twimg.com/

The next cell will actually run the bot and set it to listen to Twitter.

In [ ]:
listen()

listening
a tweet found
{'text': 'RT @KARDASHlANTREND: Kim Kardashian and Kylie Jenner https://t.co/ghtQOXmZf3', 'favorited': False, 'in_reply_to_user_id': None, 'retweeted': False, 'in_reply_to_user_id_str': None, 'id_str': '759086501879812097', 'created_at': 'Fri Jul 29 18:01:39 +0000 2016', 'filter_level': 'low', 'user': {'profile_image_url': 'http://pbs.twimg.com/profile_images/759002382571188226/stcw-vjy_normal.jpg', 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/3736097357/1469795246', 'screen_name': 'luannagdrsall', 'location': "Mark's", 'contributors_enabled': False, 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png', 'following': None, 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/759002382571188226/stcw-vjy_normal.jpg', 'profile_sidebar_border_color': 'C0DEED', 'profile_background_color': 'C0DEED', 'created_at': 'Tue Sep 22 06:27:25 +0000 2015', 'lang': 'it', 'id_str': '3736097357', 'default_profile': True, 'pro